In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import urllib.request as ur
import os.path
import zipfile

In [2]:
def fetch_company_name_cik_table():
    CIKs = []
    companyNames = []
    path = '.'
    files = ['cik-list.txt']
    for f in files:

          with open (f, "r") as myfile:
            for line in myfile:
                #print(line)
                values=line.split(':')
                companyNames.append(values[len(values)-3])
                CIKs.append(values[(len(values)-2)].strip('0'))
    df = pd.DataFrame({'CIK': CIKs, 'company': companyNames})
    df.to_csv('CIK-mapping.csv')
    return df


In [3]:
#!/usr/bin/env python       

class GetData:
    
    def __init__(self):
        """
        Retrieves and stores the urllib.urlopen object for a given url
        """
        
    def generate_url(self,year):
        
        #generate the url for fetching the log files for every month's first day
        number_of_months=1
        while number_of_months < 13:
            if(number_of_months <10):
                url="http://www.sec.gov/dera/data/PublicEDGAR-log-file-data/"+year+"/Qtr1/log"+year+'%02d' % number_of_months+"01.zip"
            else:
                url="http://www.sec.gov/dera/data/PublicEDGAR-log-file-data/"+year+"/Qtr1/log"+year+str(number_of_months)+"01.zip"
            number_of_months=number_of_months+1
        #temp_url=download_data("http://www.sec.gov/dera/data/Public-EDGAR-log-file-data/2016/Qtr1/log20160101.zip")
        return self.download_data("http://www.sec.gov/dera/data/Public-EDGAR-log-file-data/2003/Qtr1/log20030301.zip")
        
    def download_data(self,url):

        #fetching the zip file name from the URL
        file_name=url.split("/")

        #Downloading data if not already present in the cache
        if(os.path.exists("Part_2_log_datasets/"+file_name[8])):
            print("Already present")

        else:
            urllib.request.urlretrieve(url, "Part_2_log_datasets/"+file_name[8])
            print("Download complete")

        #unzip the file and fetch the csv file
        zf = zipfile.ZipFile("Part_2_log_datasets/"+file_name[8]) 
        csv_file_name=file_name[8].replace("zip", "csv")
        zf_file=zf.open(csv_file_name)

        #create a dataframe from the csv
        df = pd.read_csv(zf_file)
        return df
        
#fetch the year for which the user wants logs
year = input('Enter the year for which you need to fetch the log files: ')
#calling the function to generate dynamic URL
get_data_obj=GetData()
df=get_data_obj.generate_url(year)
        


Enter the year for which you need to fetch the log files: 2003
Already present


In [13]:
#convert all the integer column in int format

df['zone'] = df['zone'].astype('int')
df['cik'] = df['cik'].astype('int')
df['code'] = df['code'].astype('int')
df['idx']=df['idx'].astype('int')
df['norefer']=df['norefer'].astype('int')
df['noagent']=df['noagent'].astype('int')
df['find']=df['find'].astype('int')
df['crawler']=df['crawler'].astype('int')
print(df.head(25))

                 ip        date      time  zone      cik  \
0    129.110.39.jca  2003-03-01  00:00:00   500    97349   
1     61.115.76.jbf  2003-03-01  00:00:00   500   766351   
2     61.115.76.jbf  2003-03-01  00:00:01   500   902584   
3     61.115.76.jbf  2003-03-01  00:00:03   500   778207   
4    129.110.39.jca  2003-03-01  00:00:07   500    97349   
5     208.62.55.eib  2003-03-01  00:00:07   500    56824   
6    129.110.39.jca  2003-03-01  00:00:10   500    97349   
7     208.62.55.eib  2003-03-01  00:00:12   500    56824   
8    129.110.39.jca  2003-03-01  00:00:13   500    97349   
9   148.139.130.hhi  2003-03-01  00:00:16   500  1108205   
10   129.110.39.jca  2003-03-01  00:00:17   500    97349   
11   129.110.39.jca  2003-03-01  00:00:19   500    97349   
12    61.115.76.jbf  2003-03-01  00:00:21   500   778207   
13    67.81.137.eci  2003-03-01  00:00:21   500   857323   
14   203.200.34.ejd  2003-03-01  00:00:31   500  1125051   
15   129.110.39.jca  2003-03-01  00:00:3

In [14]:

#replacing empty strings with NaN 
df.replace(r'\s+', np.nan, regex=True)

,ip,date,time,zone,cik,accession,extention,code,size,idx,norefer,noagent,find,crawler,browser
0,129.110.39.jca,2003-03-01,00:00:00,500,97349,0000097349-01-000006,-0002.txt,200,3726.0,0,0,0,9,0,win
1,61.115.76.jbf,2003-03-01,00:00:00,500,766351,0000950134-03-003149,.txt,200,995957.0,0,1,0,0,0,NaN
2,61.115.76.jbf,2003-03-01,00:00:01,500,902584,0000902584-03-000044,.txt,200,15520.0,0,1,0,0,0,NaN
3,61.115.76.jbf,2003-03-01,00:00:03,500,778207,9999999997-03-006003,.txt,200,1670.0,0,1,0,0,0,NaN
4,129.110.39.jca,2003-03-01,00:00:07,500,97349,0000097349-01-000006,-index.htm,200,4331.0,1,0,0,1,0,win
5,208.62.55.eib,2003-03-01,00:00:07,500,56824,0000950124-03-000077,-index.htm,200,2727.0,1,0,0,1,0,win
6,129.110.39.jca,2003-03-01,00:00:10,500,97349,0000097349-01-000006,-0003.txt,200,1211.0,0,0,0,9,0,win
7,208.62.55.eib,2003-03-01,00:00:12,500,56824,0000950124-03-000077,k73883e10vqza.txt,200,158260.0,0,0,0,9,0,win
8,129.110.39.jca,2003-03-01,00:00:13,500,97349,0000097349-01-000006,-index.htm,200,4331.0,1,0,0,1,0,win
9,148.139.130.hhi,2003-03-01,00:00:16,500,1108205,0000927016-02-005853,-index.htm,304,NaN,1,0,0,1,0,win


In [17]:

#replace all ip column NaN value by a default ip address 
df["ip"].fillna("255.255.255.255", inplace=True)

#perform forward fill to replace NaN values by fetching the next valid value
df["date"].fillna(method='ffill')

#perform backward fill to replace NaN values by backpropagating and fetching the previous valid value
df["time"].fillna(method='bfill')

#
#df["zone"].fillna(?????)

#replace all extension column NaN values by default extension
df["extention"].fillna("-index.htm", inplace=True)

#replace all size column NaN values by 0 and convert the column into integer 
df["size"].fillna(0, inplace=True)
df['size'] = df['size'].astype('int')

#replace all user agent column NaN values by the default value 1 (no user agent)
df["noagent"].fillna("Not Applicable", inplace=True)

#replace all find column NaN values by the default value 0 (no character strings found)
df["find"].fillna(0, inplace=True)

#replace all broser column NaN values by a string
df["browser"].fillna("Not Available", inplace=True)

In [22]:
# if the value in idx column is missing, check the value of the extension column, if its "-index.html" set the column's value 1 else 0
count=0
for i in df['idx']:
    if(np.isnan(i)):
        if(df['extension'][count]=="-index.htm"):
            i=1
        else:
            i=0
    count=count+1

# if the value of norefer column is missing, check the value of the find column, if it is 0, set the value 1, else it set it 0
counter=0
for i in df['norefer']:
    if(np.isnan(i)):
        if(df["find"][counter]==0):
            i=1
        else:
            i=0
    counter=counter+1
    
# if the value of crawler is missing, check the value of the code, if it is 404 set it as 1 else 0
count_position=0
for i in df['crawler']:
    if(np.isnan(i)):
        if(df["code"][count_position]==404):
            i=1
        else:
            i=0
    count_position=count_position+1

In [25]:
#insert a column to check CIK, Accession number discripancy
df.insert(6, "CIK_Accession_Anamoly_Flag", "N")

In [26]:

#check if CIK and Accession number match. The Accession number is divided into three parts, CIK-Year-Number_of_filings_listed.
#the first part i.e the CIK must match with the CIK column. If not, there exists an anomaly

count=0;
print("I am working")
for i in df['accession']:
    #fetch the CIK number from the accession number and convert it into integer
    list_of_fetched_cik_from_accession=[(int(i.split("-")[0]))]
    
    #check if the CIK number from the column and CIK number fetched from the accession number are equal
    if(df['cik'][count]!=list_of_fetched_cik_from_accession):
        df['CIK_Accession_Anamoly_Flag'][count]="Y"
        
    count=count+1
print("Done")
print(df.head(10))

I am working


C:\Users\Vasanti\Anaconda3\lib\site-packages\ipykernel\__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Done
                ip        date      time  zone      cik             accession  \
0   129.110.39.jca  2003-03-01  00:00:00   500    97349  0000097349-01-000006   
1    61.115.76.jbf  2003-03-01  00:00:00   500   766351  0000950134-03-003149   
2    61.115.76.jbf  2003-03-01  00:00:01   500   902584  0000902584-03-000044   
3    61.115.76.jbf  2003-03-01  00:00:03   500   778207  9999999997-03-006003   
4   129.110.39.jca  2003-03-01  00:00:07   500    97349  0000097349-01-000006   
5    208.62.55.eib  2003-03-01  00:00:07   500    56824  0000950124-03-000077   
6   129.110.39.jca  2003-03-01  00:00:10   500    97349  0000097349-01-000006   
7    208.62.55.eib  2003-03-01  00:00:12   500    56824  0000950124-03-000077   
8   129.110.39.jca  2003-03-01  00:00:13   500    97349  0000097349-01-000006   
9  148.139.130.hhi  2003-03-01  00:00:16   500  1108205  0000927016-02-005853   

  CIK_Accession_Anamoly_Flag          extention  code    size  idx  norefer  \
0                       

In [9]:
#fetch the CIK_CompanyName concatenated dataframe
cik_companyname_dataframe=fetch_company_name_cik_table()
cik_companyname_dataframe = cik_companyname_dataframe.rename(columns={'CIK': 'cik'})
print(cik_companyname_dataframe.head(5))


       cik                             company
0  1438823                              !J INC
1  1509607       #1 A LIFESAFER HOLDINGS, INC.
2  1457512  #1 ARIZONA DISCOUNT PROPERTIES LLC
3  1433777                   #1 PAINTBALL CORP
4  1427189                               $ LLC


In [10]:
#merge both the dataframe using the CIK as common column
# <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< facing a problem, both the cik's are different>>>>>>>>>>>>>>>>>>>>>>
print(cik_companyname_dataframe.sort(['cik'], ascending=[False]).head(50))
print(df.sort(['cik'], ascending=[False]).head(50))

#merged_df=df.merge(cik_companyname_dataframe, on='cik', how='left')
#print(merged_df.head(25))
#print(merged_df.loc[merged_df['cik']==1438823])



C:\Users\Vasanti\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


          cik                                            company
593019  99982                             TRUST CO OF NEW JERSEY
583843  99974                               TJ INTERNATIONAL INC
592537  99947                        TRUBEE, COLLINS & CO., INC.
592458  99927                               TROY INVESTMENT FUND
592303  99889                             TROPICANA PRODUCTS INC
53910    9984                                   BARNES GROUP INC
591818   9983                                 TRITON ENERGY CORP
591687  99817                                  TRIPP & CO., INC.
591686  99817  TRIPP & CO INC                                ...
591528  99802                                          TRION INC
591387  99782                 TRINITY INVESTMENT MANAGEMENT CORP
591384   9978                             TRINITY INDUSTRIES INC
591322  99771                               TRINITY CAPITAL CORP
590801  99724                                TRICO PRODUCTS CORP
590581  99703            

C:\Users\Vasanti\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [11]:
print(df.head(10))

                ip        date      time  zone      cik             accession  \
0   129.110.39.jca  2003-03-01  00:00:00   500    97349  0000097349-01-000006   
1    61.115.76.jbf  2003-03-01  00:00:00   500   766351  0000950134-03-003149   
2    61.115.76.jbf  2003-03-01  00:00:01   500   902584  0000902584-03-000044   
3    61.115.76.jbf  2003-03-01  00:00:03   500   778207  9999999997-03-006003   
4   129.110.39.jca  2003-03-01  00:00:07   500    97349  0000097349-01-000006   
5    208.62.55.eib  2003-03-01  00:00:07   500    56824  0000950124-03-000077   
6   129.110.39.jca  2003-03-01  00:00:10   500    97349  0000097349-01-000006   
7    208.62.55.eib  2003-03-01  00:00:12   500    56824  0000950124-03-000077   
8   129.110.39.jca  2003-03-01  00:00:13   500    97349  0000097349-01-000006   
9  148.139.130.hhi  2003-03-01  00:00:16   500  1108205  0000927016-02-005853   

  CIK_Accession_Anamoly_Flag          extention  code      size  idx  norefer  \
0                          

In [28]:
df.insert(7, "filename", "")

In [ ]:
#Extension rule: if the file name is missing and only the file extension is present, then the file name is document accession number
count=0
for i in df["extention"]:
    if(i==".txt"):
        # if the value in extension is only .txt, fetch the accession number and append accession number to .txt
        #list_of_fetched_cik_from_accession=int(((df2["accession"].str.split("-")[count])[0]))
        #print((df["accession"]).astype(str))
        #list_of_fetched_cik_from_accession=int(df["accession"])
        df["filename"][count]=(df["accession"]).astype(str)+".txt"
    else:
        df["filename"][count]=i
    count=count+1
print(df.head(10))

C:\Users\Vasanti\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Vasanti\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
